In [1]:
from __future__ import print_function
import os
import numpy as np
from numpy import newaxis
from sklearn import preprocessing
np.random.seed(1337)

import keras
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Dropout
from keras.models import Model,Sequential
import sys

import pandas as pd
import random

Using TensorFlow backend.


# Reading in data

In [2]:
print("Data read started...")
data = pd.read_csv("result_nback.csv")
data = data.as_matrix()
print ("Data read finished.")

print(data.shape)

Data read started...
Data read finished.
(828, 11)


# Eliminate EEG data

In [3]:
for i in range (1,5):
    data = np.delete(data, i, 1) 

data.shape

(828, 7)

In [4]:
data[1]

array(['nback gyakorlas', 286880.0, 170520.0, 933.41, 4.9275, 307.67,
       343.79], dtype=object)

# Dictionary for the levels

In [5]:
level=["nback gyakorlas","0back","1back","2back","3back"]
#level=["tetris1","tetris2","tetris3"]
level2int = dict((p, i) for i, p in enumerate(level))
int2level = dict((i, p) for i, p in enumerate(level))

# Z normalize

In [6]:
for i in range (1,7):
    data[:, i] = preprocessing.scale(data[:, i])

/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConve

In [7]:
for i in range(len(data)):
    data[i, 0] = level2int[data[i, 0]]

x_data = data[:, 1:]
y_data = data[:, 0]

# One-hot encoding

In [8]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
y_one_hot = ohe.fit_transform(y_data.reshape(-1,1)).toarray()
print(y_one_hot)

[[ 1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.]
 ..., 
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.]]


# Shuffle data

In [9]:
indices = np.arange(x_data.shape[0])
np.random.shuffle(indices)

x_data = x_data[indices]
y_one_hot = y_one_hot[indices]

# Divide into train, validation and test data

In [10]:
len_data = len(x_data)

nb_test = int(len_data*0.15)
nb_validation = int(len_data*0.15)
nb_train = int(len_data*0.7)

end_valid = nb_train+nb_validation

x_train = x_data[0:nb_train]
y_train = y_one_hot[0:nb_train]

x_valid = x_data[nb_train:end_valid]
y_valid = y_one_hot[nb_train:end_valid]

x_test = x_data[end_valid:]
y_test = y_one_hot[end_valid:]

In [11]:
print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape, x_test.shape, y_test.shape)

(579, 6) (579, 5) (124, 6) (124, 5) (125, 6) (125, 5)


# Build the net

In [14]:
model = Sequential()

model.add(Dense(200, input_shape=(6,)))
model.add(Dropout(0.15))
model.add(Dense(200, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(5, activation='softmax'))

In [15]:
batch_size = 50
epochs = 150
earlyStopping=keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, verbose=0, mode='auto')

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(x_train, y_train, nb_epoch=epochs,batch_size=batch_size, 
          callbacks=[earlyStopping], shuffle=True, validation_data = (x_valid, y_valid))

Train on 579 samples, validate on 124 samples
Epoch 1/150
579/579 [==============================] - 0s - loss: 1.4965 - acc: 0.3523 - val_loss: 1.3735 - val_acc: 0.3629
Epoch 2/150
579/579 [==============================] - 0s - loss: 1.3574 - acc: 0.4180 - val_loss: 1.2880 - val_acc: 0.4597
Epoch 3/150
579/579 [==============================] - 0s - loss: 1.2925 - acc: 0.4698 - val_loss: 1.2478 - val_acc: 0.4597
Epoch 4/150
579/579 [==============================] - 0s - loss: 1.2419 - acc: 0.4750 - val_loss: 1.1938 - val_acc: 0.5161
Epoch 5/150
579/579 [==============================] - 0s - loss: 1.2052 - acc: 0.4991 - val_loss: 1.1590 - val_acc: 0.5000
Epoch 6/150
579/579 [==============================] - 0s - loss: 1.1692 - acc: 0.5181 - val_loss: 1.0899 - val_acc: 0.4919
Epoch 7/150
579/579 [==============================] - 0s - loss: 1.1251 - acc: 0.5302 - val_loss: 1.0904 - val_acc: 0.5565
Epoch 8/150
579/579 [==============================] - 0s - loss: 1.0903 - acc: 0.5492

In [16]:
model.evaluate(x_test, y_test, batch_size=batch_size)

125/125 [==============================] - 0s     


[0.60787882208824162, 0.78400000333786013]

In [17]:
from sklearn.metrics import classification_report,confusion_matrix

y_pred = model.predict_classes(x_test)
print(y_pred)

p=model.predict_proba(x_test)

target_names = ['class 0(nback gyakorlas)', 'class 1(0back)', 'class 2(1back)', 'class 3(2back)', 'class 4(3back)']
print(classification_report(np.argmax(y_test,axis=1), y_pred,target_names=target_names))
print(confusion_matrix(np.argmax(y_test,axis=1), y_pred))

125/125 [==============================] - 0s     
[2 3 3 3 4 3 1 4 2 2 4 0 4 0 1 3 2 4 4 4 4 3 4 4 3 3 2 1 2 4 1 4 0 2 4 0 1
 0 3 0 1 4 1 2 0 4 3 4 4 4 3 4 0 2 4 0 3 4 2 0 1 0 3 2 4 4 2 2 2 2 1 4 4 4
 4 3 4 3 2 4 1 4 4 1 4 4 2 1 2 2 2 2 4 4 4 2 3 4 1 3 0 4 2 3 0 4 0 4 0 3 0
 4 4 3 2 0 0 0 2 3 3 4 3 4 0]
 32/125 [======>.......................] - ETA: 0s                          precision    recall  f1-score   support

class 0(nback gyakorlas)       0.90      0.90      0.90        20
          class 1(0back)       0.69      0.47      0.56        19
          class 2(1back)       0.60      0.75      0.67        20
          class 3(2back)       0.78      0.67      0.72        27
          class 4(3back)       0.86      0.97      0.92        39

             avg / total       0.78      0.78      0.78       125

[[18  0  2  0  0]
 [ 0  9  7  3  0]
 [ 0  3 15  1  1]
 [ 2  1  1 18  5]
 [ 0  0  0  1 38]]
